# Regression Model B: Field 3

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 3 (F3)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E3" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E3" ),]
dE4=d4[which(d4$Estate=="E3" ),]
dE5=d5[which(d5$Estate=="E3" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
14,1,1,1,1,1,1,1,1,1,1,1,0
13,1,1,1,1,1,1,1,1,1,0,0,2
8,1,1,1,1,1,1,1,0,0,1,1,2
9,1,1,1,1,1,1,1,0,0,0,0,4
,0,0,0,0,0,0,0,17,17,22,22,78


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost
  3   3  prun.time  prun.cost  manuring.time  manuring.cost
  3   4  prun.time  prun.cost  manuring.time  manuring.cost
  3   5  prun.time  prun.cost  manuring.time  manuring.cost
  4   1  prun.time  prun.cost  manuring.time  manuring.cost
  4   2  prun.time  

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            22            22            17            17 
      pd.time       pd.cost 
            0             0 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost 
            2             3             4             5 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1             1             1       1
prun.cost          1         1  

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper",col=brewer.pal(n=8, name="RdYlBu"))


## Regression

Fit the data using multiple regression for all variables. 

In [10]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=df2)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-36.224 -21.638  -4.794  16.475  72.871 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)   
(Intercept)    1.346e+02  3.944e+01   3.413  0.00164 **
rainfall      -3.209e-03  7.455e-03  -0.430  0.66950   
foliar        -3.236e+00  1.445e+00  -2.239  0.03161 * 
prun.time     -4.093e+00  8.269e+00  -0.495  0.62370   
prun.cost      5.467e-03  5.189e-03   1.054  0.29926   
manuring.time -1.240e+01  8.160e+00  -1.520  0.13757   
manuring.cost  1.967e-03  6.998e-04   2.811  0.00804 **
pd.time       -5.698e+00  6.554e+00  -0.869  0.39051   
pd.cost        4.944e-03  6.209e-03   0.796  0.43120   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 29.6 on 35 degrees of freedom
Multiple R-squared:  0.3306,	Adjusted R-squared:  0.1776 
F-statisti

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [11]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,341.20667,341.20667,0.38948846,0.536615323
foliar,1,6795.98322,6795.98322,7.75763575,0.008574932
prun.time,1,14.85724,14.85724,0.01695959,0.897130880
prun.cost,1,230.30255,230.30255,0.26289107,0.611361860
manuring.time,1,549.09925,549.09925,0.62679848,0.433865157
manuring.cost,1,6268.78298,6268.78298,7.15583506,0.011284354
pd.time,1,384.71962,384.71962,0.43915863,0.511869594
pd.cost,1,555.54819,555.54819,0.63415997,0.431202126
Residuals,35,30661.32779,876.03794,NA,NA


### Using stepwise

In [12]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=306.05
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- rainfall       1     162.3 30824 304.28
- prun.time      1     214.6 30876 304.36
- pd.cost        1     555.5 31217 304.84
- pd.time        1     662.3 31324 304.99
- prun.cost      1     972.6 31634 305.42
<none>                       30661 306.05
- manuring.time  1    2023.2 32685 306.86
- foliar         1    4392.1 35053 309.94
- manuring.cost  1    6920.1 37581 313.00

Step:  AIC=304.28
yield ~ foliar + prun.time + prun.cost + manuring.time + manuring.cost + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- prun.time      1     152.2 30976 302.50
- pd.cost        1     562.7 31386 303.08
- pd.time        1     639.9 31464 303.19
- prun.cost      1    1095.5 31919 303.82
<none>                       30824 304.28
- manuring.time  1    2152.9 32977 305.25
- foliar         1    4252.8 35076 307.97
-


Call:
lm(formula = yield ~ foliar + manuring.time + manuring.cost, 
    data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-35.223 -23.995  -1.793  17.146  65.911 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.161e+02  1.372e+01   8.461 1.89e-10 ***
foliar        -3.384e+00  1.282e+00  -2.639  0.01179 *  
manuring.time -1.096e+01  7.523e+00  -1.457  0.15284    
manuring.cost  1.791e-03  6.479e-04   2.765  0.00857 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.56 on 40 degrees of freedom
Multiple R-squared:  0.2875,	Adjusted R-squared:  0.2341 
F-statistic:  5.38 on 3 and 40 DF,  p-value: 0.00331


       foliar manuring.time manuring.cost 
     1.002185      1.038941      1.041106 



Call:
lm(formula = yield ~ foliar + manuring.time + manuring.cost, 
    data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-35.223 -23.995  -1.793  17.146  65.911 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.161e+02  1.372e+01   8.461 1.89e-10 ***
foliar        -3.384e+00  1.282e+00  -2.639  0.01179 *  
manuring.time -1.096e+01  7.523e+00  -1.457  0.15284    
manuring.cost  1.791e-03  6.479e-04   2.765  0.00857 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.56 on 40 degrees of freedom
Multiple R-squared:  0.2875,	Adjusted R-squared:  0.2341 
F-statistic:  5.38 on 3 and 40 DF,  p-value: 0.00331


## Prediction

Predict with the resample values.

In [13]:
set.seed(123)
df3=sample(1:nrow(df2),12)
df4=df2[df3,]
estimate=predict(selectedMod,newdata=df4)
estimate
sum.estimate=sum(estimate)
sum.estimate

df5=data.frame(estimate,sum.estimate)

13        34        18        37        38         2        21        43 
105.28453  76.49969 121.06925 121.05725 103.57411 103.58909  69.07907  77.07423 
       20        16        33        15 
 82.87840 132.48051  87.89076 111.05131

[1] 1191.528